# 8 WorkFlow  Semillerio

### 8.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 8.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



*  Si usted eligió modalidad *Gerencial* entonces NO debe modificar nada, su liderazgo logró que otras trabajaron por usted
*  Si usted eligió modalidad de *Analista Junior*, entonces donde dice archivo="gerencial_competencia_2025.csv.gz"  lo debe cambiar por  archivo="analistajr_competencia_2025.csv.gz"
*  Si usted eligió modalidad *Analista Senior*,  ya se las ingenierá SIN preguntar !

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 8.3  Workflow Semillerio

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

### Usted DEBE ambiar "PARAM experimento" cada vez que desee correr un nuevo experimento

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 314159

PARAM$experimento <- "-clon"
PARAM$dataset <- "analistasr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 8.3.1   Preprocesamiento del dataset

#### 8.3.1.1  DT incorporar dataset

In [ ]:
# lectura del dataset
dataset <- fread(paste0("/content/buckets/b1/datasets/", PARAM$dataset))

#### 8.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [ ]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 8.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [ ]:
# sin codigo en esta primera version del workflow

#### 8.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [ ]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


In [ ]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

#### 8.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [ ]:
# No se implementa Feature Engineering a partir de Random Forest

#### 8.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [ ]:
# Feature Engineering Historico
# ordeno el dataset
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [ ]:
ncol(dataset)
colnames(dataset)

#### 8.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [ ]:
# No se implementa la reduccion de la dimensionalidad con canaritos

## Algoritmo Genético

### Requirements

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

if(!require("lightgbm")) install.packages("lightgbm")
        require("lightgbm")

### Selección de variables de entrada

In [ ]:
#Defino los campos

campos <- colnames(dataset)
campos_monetarios <- campos[campos %like%
  "^(m|Visa_m|Master_m|vm_m)"]
campos_cant <- campos[campos %like% "^c" & campos != "cliente_vip"]

In [ ]:
# Defino pool inicial de variables

if (!file.exists("var_pool.txt")){
  var_pool <- data.table(
    Feature = cols_lagueables, #Variable
    from1 = NA, #variables 1 usada en iteración anterior
    from2 = NA, #variables 2 usada en iteración anterior
    op = NA, #operación usada para obtenerla
    complete_form = cols_lagueables #forma completa (para uso final)
  )

  var_pool[, clase := fifelse(Feature %in% campos_monetarios, "m",
                      fifelse(Feature %in% campos_cant, "c", "b"))]
} else {
  var_pool <- fread("var_pool.txt")
}

### Params

In [ ]:
meses <- data.frame(dataset$foto_mes)
meses_unique <- as.list(sapply(meses, unique))

In [ ]:
PARAM$trainingstrategy$validate <- c(202105, 202104)

PARAM$trainingstrategy$training <- c(
  202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005, 202004, 202003
)

PARAM$trainingstrategy$training_pct <- 0.1


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
PARAM$hipeparametertuning$num_interations <- 50

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  extra_trees = FALSE,

  max_depth = -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split = 0.0, # min_gain_to_split >= 0.0
  min_sum_hessian_in_leaf = 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1 = 0.0, # lambda_l1 >= 0.0
  lambda_l2 = 0.0, # lambda_l2 >= 0.0

  bagging_fraction = 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction = 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction = 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance = FALSE, #
  scale_pos_weight = 1.0, # scale_pos_weight > 0.0

  drop_rate = 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop = 50, # <=0 means no limit
  skip_drop = 0.5, # 0.0 <= skip_drop <= 1.0

  max_bin= 31,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)


PARAM$hipeparametertuning$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.02, upper = 0.3),
  makeNumericParam("feature_fraction", lower = 0.05, upper = 0.90),
  makeNumericParam("coverage", lower = 0.05, upper = 1.0), # nuevo
  makeNumericParam("leaf_size", lower = 0.001, upper = 0.2) # nuevo
)

In [ ]:
PARAM$trainingstrategy$final_train <- c( 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007, 202006, 202005, 202004, 202003
)


In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

### Funciones

In [ ]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  message(format(Sys.time(), "%a %b %d %X %Y"))

  # uno la lista de hiperparametros : fijos + variables
  param_completo <- c(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  return(AUC)
}

get_combinations <- function(cols, n, seed) {
  set.seed(seed)
  combinations <- t(combn(cols, 2))
  if (nrow(combinations) > n) {
    return(combinations[sample(nrow(combinations), n), ])
  } else {
    return(combinations)
  }
}

### Utils

In [ ]:
safe_ops <- list(
  `*`    = function(a, b) a * b,
  `/`    = function(a, b) a / b
)

all_ops <- list(
  `+`    = function(a, b) a + b,
  `-`    = function(a, b) a - b,
  `*`    = function(a, b) a * b,
  `/`    = function(a, b) a / b
)

### Algoritmo Genético

Setup

In [ ]:
# Numero de generaciones
n_generations <- 25

# Numero de mutaciones a realizar
n_mutations <- 100

In [ ]:
# Loop principal

# guardo dataset original en otra variable
dataset_original <- copy(dataset)

for (generation in 1:n_generations) {

    if(exists("dataset")){rm(dataset)} #por las dudas
    dataset <- copy(dataset_original) #(para no tener que ir dropeando variables que se van, ver si es bueno para la memoria)

    impo_name <- paste0("impo_",generation,".txt")
    seed_loop <- PARAM$semilla_primigenia + generation - 1 #por ahora generamos las seed sumando de a una
    #check generacion completa
    if (file.exists(impo_name)){ 
        next #si existe el impo de esta generación quiere decir que ya se procesó todo
    }

    #cargo var_pool
    cols_to_mutate <- var_pool[,Feature]

    #si no hay alguna variable la creo
    for (var in cols_to_mutate){
                # Checkeo si existe en el dataset
                if(!(var %in% colnames(dataset)))
                    {
                    dataset <- dataset[, (var) := get(complete_form)]  #creo que esto no anda aun, tengo que traerme la var
                    }
            }

    # deletear cosas que no sirven para no saturar la memoria
    if(exists("dtrain")){rm(dtrain)}
    if(exists("dvalidate")){rm(dvalidate)}
    if(exists("dfinal_train")){rm(dfinal_train)}
    if(exists("tb_importancia")){rm(tb_importancia)}    

    # crear variables nuevas
        # combinaciones
        # mutaciones
        # guaradar provisoriamente las variables nuevas (solo guardamos en var_pool las que realmente nos van a servir)

    combinations <- get_combinations(cols_to_mutate, n_mutations, seed_loop)

    new_vars <- data.table(
                    Feature = character(),
                    from1 = character(),
                    from2 = character(),
                    op = character(),
                    complete_form = character(),
                    clase = character()
                )

    for (var_num in 1:n_mutations){
        var_name <- paste0("gen_",generation,"_n_",var_num)
        var1 <- combinations[var_num, 1]
        var2 <- combinations[var_num, 2]
        clase1 <- var_pool[Feature == var1, clase][[1]]
        clase2 <- var_pool[Feature == var2, clase][[1]]
        if (!is.na(clase1) && !is.na(clase2) &&
            ((clase1 == "m" && clase2 == "m") ||
            (clase1 == "c" && clase2 == "c") ||
            (clase1 == "b" && clase2 == "b"))) {
            safe <- FALSE
            clase_op <- clase1
        } else {
            safe <- TRUE
            clase_op <- NA
        }

        # elijo una operación
        inline_seed <- seed_loop*var_num
        set.seed(inline_seed)
        operation_name <- ifelse(safe == TRUE, sample(names(safe_ops), 1), sample(names(all_ops), 1))
        operation <- all_ops[[operation_name]] #como acá estan todas por ahora puedo esar siempre este

        # realizo la mutación
        dataset <- dataset[, (var_name) := operation(get(var1),get(var2))]
        complete_form_1 = var_pool[Feature == var1, complete_form][[1]]
        complete_form_2 = var_pool[Feature == var2, complete_form][[1]]
        new_var <- data.table(
        Feature = var_name,
        from1 = combinations[var_num,1],
        from2 = combinations[var_num,2],
        op = operation_name,
        complete_form = paste0("(",complete_form_1,operation_name,complete_form_2,")"),
        clase = clase_op
        )

        # la agrego a las variables recien creadas para analizar luego
        new_vars <- rbind(new_vars, new_var)
    }

    ######## bayesiana

    # los campos en los que se entrena
    campos_buenos <- copy( setdiff(
        colnames(dataset), c("clase_ternaria","clase01","azar")))

    # preparo para que se puede hacer undersampling de los CONTINUA
    #  solamente por un tema de VELOCIDAD
    set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
    dataset[, azar:=runif(nrow(dataset))]

    # undersampling de los CONTINUA
    dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
        (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
        azar < PARAM$trainingstrategy$training_pct ) ]

    dtrain <- lgb.Dataset(
        data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
        label= dataset[fold_train == TRUE, clase01],
        free_raw_data= TRUE
        )

    # datos de validation
    dvalidate <- lgb.Dataset(
    data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
    label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
    free_raw_data= TRUE
    )


    # Esto podría sacarlo pero por ahora lo dejo por el ctrl
    configureMlr(show.learner.output = FALSE)

    # configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
    # por favor, no desesperarse por lo complejo
    obj.fun <- makeSingleObjectiveFunction(
        fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
        minimize= FALSE, # estoy Maximizando AUC
        noisy= FALSE,
        par.set= PARAM$hipeparametertuning$hs,
        has.simple.signature= FALSE # paso los parametros en una lista
    )

    # cada 600 segundos guardo el resultado intermedio
    save_path <- paste0("HT_",generation,".RDATA")
    ctrl <- makeMBOControl(
        save.on.disk.at.time= 600,
        save.file.path= save_path
    )

    # indico la cantidad de iteraciones que va a tener la Bayesian Optimization
    ctrl <- setMBOControlTermination(
        ctrl,
        iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
    )

    # defino el método estandar para la creacion de los puntos iniciales
    #   los "No Inteligentes"
    ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

    # mas configuraciones
    surr.km <- makeLearner(
        "regr.km",
        predict.type= "se",
        covtype= "matern3_2",
        control= list(trace = TRUE)
    )

    # inicio la optimizacion bayesiana

    if (!file.exists(save_path)) {
    bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
    } else {
    bayesiana_salida <- mboContinue(save_path) # retomo en caso que ya exista
    }

    # almaceno los resultados de la Bayesian Optimization
    # y capturo los mejores hiperparametros encontrados

    tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

    # ordeno en forma descendente por AUC = y
    setorder(tb_bayesiana, -y, -num_iterations)

    # grabo para eventualmente poder utilizarlos en OTRA corrida
    file_path <- paste0("BO_",generation,"_log.txt")
    fwrite( tb_bayesiana,
    file=file_path,
    sep="\t"
    )

    # los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
    PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
    1, # el primero es el de mejor AUC
    list(learning_rate, feature_fraction, coverage, leaf_size)
    ]

    print(PARAM$out$lgbm$mejores_hiperparametros)

    ######## final train

    dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

    # creo el dfinal_train en formato  LightGBM
    dfinal_train <- lgb.Dataset(
    data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
    label= dataset[fold_final_train == TRUE, clase01],
    free_raw_data= TRUE
    )

    nrow( dfinal_train) # verifico el tamaño


    # uno los parametros fijos y los mejores encontrados de los variables
    fijos <- copy(PARAM$lgbm$param_fijos)

    # quito lo que optimice en la Bayesian Optimization
    fijos$num_iterations <- NULL
    fijos$early_stopping_rounds <- NULL

    # agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
    param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

    # hago la transformacion de leaf_size y  coverage
    if( "leaf_size"  %in% names(param_final) &
        "coverage"  %in% names(param_final)
    )
    {
        # primero defino el tamaño de las hojas
        param_final$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_final$leaf_size )  )
        # luego la cantidad de hojas en funcion del valor anterior, el coverage, y la cantidad de registros
        param_final$num_leaves <- pmin( 131072,
        pmax( 8,  round( ( param_final$coverage * nrow( dtrain ) / param_final$min_data_in_leaf ) ) ))
    }

    if( "leaf_size"  %in% names(param_final) &
        !( "coverage"  %in% names(param_final) )
    )
    {
        # primero defino el tamaño de las hojas
        param_final$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_final$leaf_size )  )
    }
                        
    final_model <- lgb.train(
    data= dfinal_train,
    param= param_final,
    verbose= -100
    )

    modelo_path <- paste0("modelo_",generation,".txt")
    lgb.save(final_model, modelo_path)


    # ahora imprimo la importancia de variables

    tb_importancia <- as.data.table(lgb.importance(final_model))
    archivo_importancia <- impo_name

    fwrite( tb_importancia,
    file= archivo_importancia,
    sep= "\t"
    )

    #### comparación de las variables con sus padres
    tb_importancia[, rank := .I]
    

    for (i in 1:nrow(new_vars)){
        fila <- new_vars[i]

        var_rank <- tb_importancia[Feature == fila$Feature, if (.N == 0) NA else rank[[1]]]
        from1_rank <- tb_importancia[Feature == fila$from1, if (.N == 0) NA else rank[[1]]]
        from2_rank <- tb_importancia[Feature == fila$from2, if (.N == 0) NA else rank[[1]]]

        if (length(var_rank) == 0) var_rank <- NA
        if (length(from1_rank) == 0) from1_rank <- NA
        if (length(from2_rank) == 0) from2_rank <- NA

        if (is.na(var_rank)){
            next
        }

        if (is.na(from1_rank) & is.na(from2_rank)){ #significa que es mas significativo var_rank porque no es na
            var_pool <- rbind(var_pool, fila)
        }

        if (!is.na(var_rank) && !is.na(from1_rank) && !is.na(from2_rank)) {
            if ((var_rank < from1_rank) & (var_rank < from2_rank)) {
                var_pool <- rbind(var_pool, fila)
            }
        }

    }

    fwrite( new_vars,
        file= "new_vars.txt",
        sep= "\t"
        )

    fwrite(var_pool,
           file = "var_pool.txt",
           sep = "\t"
          )
    

}

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")